<a href="https://colab.research.google.com/github/darshbaxi/Emoji_prediction/blob/main/Emoji_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.preprocessing.text import one_hot

In [17]:
df=pd.read_csv('Train.csv')


In [18]:
df.head()

,Unnamed: 0,TEXT,Label
0,0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,2,Been friends since 7th grade. Look at us now w...,2
3,3,This is what it looks like when someone loves ...,3
4,4,RT @user this white family was invited to a Bl...,3


In [19]:
df=df.iloc[:,1:]

In [ ]:

df.head(25)

In [21]:
y=df['Label']

In [22]:
df=df.iloc[:,:1]

In [23]:
x=df['TEXT']

In [24]:

import nltk


from nltk.stem import PorterStemmer as ps
from nltk.stem import WordNetLemmatizer as wl
from nltk.corpus import stopwords

lammtizer=wl()
nltk.download('wordnet')
nltk.download('stopwords')
stemmer=ps()
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [25]:
import re
fin=[]
for i in range(len(df)):
  review=re.sub('[^a-zA-Z]',' ',x[i])
  review=review.lower()
  word=nltk.word_tokenize(review)
  arr=[]
  for j in word:
    if j not in set(stopwords.words('english')):
      arr.append(lammtizer.lemmatize(j))
  review=' '.join(arr)
  fin.append(review)

In [ ]:
fin

In [27]:
x_train=pd.DataFrame(fin)

In [ ]:
x_train

In [29]:
map=pd.read_csv('Mapping.csv')


In [ ]:

map=map.iloc[:,1:]
map

In [31]:
for j in range(0,70000):
  i=0
  for i in map['number']:
    if(y[j]==map['number'][i]):
      y[j]=map['emoticons'][i]

In [32]:
y

0        😜
1        📸
2        😍
3        😂
4        😂
        ..
69995    😂
69996    😎
69997    😍
69998    😂
69999    💙
Name: Label, Length: 70000, dtype: object

In [34]:
file=open('/content/glove.6B.300d.txt','r',encoding='utf8')
content=file.readlines()
file.close()

In [ ]:
content

In [36]:
embeding={}


for i in content:
  i=i.split()
  embeding[i[0]]=np.array(i[1:])

In [ ]:
embeding

In [39]:
from keras.preprocessing.text import Tokenizer

In [51]:
onrhot=[one_hot(word,10000) for word in fin]

In [ ]:
onrhot

In [55]:
def get_maxlen(fina):
    maxlen = 0
    for sent in fina:
        maxlen = max(maxlen, len(sent))
    return maxlen

max_len = get_maxlen(fin)
print(max_len)

127


In [56]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [57]:
embedded_docs=pad_sequences(onrhot,padding='post',maxlen=max_len)

In [58]:
embedded_docs

array([[1010, 3657, 1010, ...,    0,    0,    0],
       [3968, 3567, 8226, ...,    0,    0,    0],
       [ 661, 1256, 8626, ...,    0,    0,    0],
       ...,
       [ 307, 7646, 5797, ...,    0,    0,    0],
       [9423, 3065, 7748, ...,    0,    0,    0],
       [5573,   29, 2938, ...,    0,    0,    0]], dtype=int32)

In [61]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [62]:
model=Sequential()
model.add(Embedding(10001,300,input_length=max_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='CategoricalCrossentropy(from_logits=True)',optimizer='adam',metrics=['accuracy'])

In [63]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 127, 300)          3000300   
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 3,160,801
Trainable params: 3,160,801
Non-trainable params: 0
_________________________________________________________________
